# <u><h><center> Pauta C4b</center></h></u>
## IIC2115 - Programación Como Herramienta para la Ingeniería
### Ayudante: Felipe Fuentes

#### La primera parte es para generar la base de datos, utilizada en la actividad C4a

In [1]:
import sqlite3  # importamos el módulo de sqlite

connection = sqlite3.connect('futbol.db') 
            
# Generamos un cursor al contenido de la base de datos, lo que nos permite 
# acceder al contenido de las tablas y a realizar consultas sobre ellas.   
cursor = connection.cursor()                    

# Creamos las tablas:

cursor.execute("CREATE TABLE Equipos(eid INTEGER PRIMARY KEY, nombre TEXT UNIQUE)")
cursor.execute("CREATE TABLE Temporadas(tid INTEGER PRIMARY KEY, ano INTEGER UNIQUE)")
cursor.execute("CREATE TABLE Ligas(lid INTEGER PRIMARY KEY, nombre TEXT UNIQUE)")

cursor.execute("""CREATE TABLE Partidos(pid INTEGER PRIMARY KEY, liga INTEGER, 
    temporada INTEGER, fecha DATE, equipo INTEGER, localidad TEXT, resultado TEXT,
    puntos INTEGER, goles INTEGER, goles_perdidos INTEGER, pases_profundos INTEGER, 
    pases_profundos_permitidos INTEGER, ppda REAL, oppda REAL, 
    FOREIGN KEY (liga) REFERENCES Ligas, FOREIGN KEY (temporada) REFERENCES Temporadas, 
    FOREIGN KEY (equipo) REFERENCES Equipos)""")

connection.commit()
connection.close()

OperationalError: table Equipos already exists

In [24]:
connection = sqlite3.connect('futbol.db')  
cursor = connection.cursor()                    

# Con esto podemos eliminar las tablas que creamos:

cursor.execute("DROP TABLE IF EXISTS  Equipos")
cursor.execute("DROP TABLE IF EXISTS Temporadas")
cursor.execute("DROP TABLE IF EXISTS  Ligas")
cursor.execute("DROP TABLE IF EXISTS  Partidos")

connection.commit() 
connection.close()

In [29]:
#importamos los datos

import json
with open('football_results.json', encoding = 'utf8') as movies_file:
    partidos = json.load(movies_file)


In [30]:
#insertamos los datos

connection = sqlite3.connect('futbol.db')
cursor = connection.cursor()

eid = 1
tid = 1
lid = 1
pid = 1
dicc_equipos= {}
dicc_season= {}
dicc_ligas= {}


#el orden de inserción es importante, fijarse en crear primero las tablas a las que se les 
#va a hacer referencia por medio de foreign keys
for partido in partidos:
    
    if not partido["team"] in dicc_equipos:
        cursor.execute("INSERT INTO Equipos(eid, nombre) VALUES (?,?)", (eid, partido["team"]))
        dicc_equipos[partido["team"]]= eid
        eid+=1
        
    if not partido["league"] in dicc_ligas:
        cursor.execute("INSERT INTO Ligas(lid, nombre) VALUES (?,?)", (lid, partido["league"]))
        dicc_ligas[partido["league"]]= lid
        lid+=1
            
    if not partido["season"] in dicc_season:
        cursor.execute("INSERT INTO Temporadas(tid, ano) VALUES (?,?)", (tid, partido["season"]))
        dicc_season[partido["season"]]= tid
        tid+=1
        
    cursor.execute("INSERT INTO Partidos(pid, liga, temporada, fecha, equipo, localidad, resultado, puntos, goles, goles_perdidos, pases_profundos, pases_profundos_permitidos,ppda, oppda) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)", (pid, dicc_ligas[partido["league"]], dicc_season[partido["season"]], partido["date"], dicc_equipos[partido["team"]], partido["h_a"], partido["result"], partido["pts"],partido["goals_scored"], partido["goals_missed"], partido["deep_passes"], partido["deep_passes_allowed"], partido["ppda"], partido["oppda"]))

    pid += 1

    
    


connection.commit()
connection.close()

## C4b

### Consultas

#### Consulta 1

In [31]:
connection = sqlite3.connect('futbol.db')
cursor = connection.cursor()

cursor.execute("""SELECT Ligas.nombre, COUNT(DISTINCT Partidos.equipo) FROM Ligas, Partidos
                    WHERE Ligas.lid=Partidos.liga
                    GROUP BY Partidos.liga""")
print("Tabla:\n Liga | Numero de equipos distintos")
for row in cursor.fetchall():
    print(row)
connection.close()

Tabla:
 Liga | Numero de equipos distintos
('Bundesliga', 24)
('EPL', 30)
('La_liga', 30)
('Ligue_1', 29)
('RFPL', 25)
('Serie_A', 30)


#### Consulta 2

In [34]:
connection = sqlite3.connect('futbol.db')
cursor = connection.cursor()


cursor.execute("""SELECT Equipos.nombre, Sum(Partidos.goles-Partidos.goles_perdidos) as goles, Sum(Partidos.goles), Sum(Partidos.goles_perdidos)
                    FROM Equipos, Partidos
                    WHERE Equipos.eid=Partidos.equipo
                    GROUP BY Partidos.equipo
                    Order By goles DESC""")
print("Tabla:\n Equipo | Diferencias de goles | Goles | Goles Perdidos")
for row in cursor.fetchall():
    print(row)
connection.close()

Tabla:
 Equipo | Diferencias de goles | Goles | Goles Perdidos
('Barcelona', 423, 613, 190)
('Paris Saint Germain', 386, 556, 170)
('Bayern Munich', 380, 529, 149)
('Manchester City', 334, 537, 203)
('Real Madrid', 333, 561, 228)
('Juventus', 288, 456, 168)
('Liverpool', 218, 451, 233)
('Napoli', 216, 456, 240)
('Atletico Madrid', 212, 364, 152)
('Zenit St. Petersburg', 201, 337, 136)
('Roma', 194, 431, 237)
('Borussia Dortmund', 182, 430, 248)
('Tottenham', 179, 415, 236)
('Lyon', 177, 418, 241)
('Chelsea', 162, 411, 249)
('CSKA Moscow', 161, 303, 142)
('Inter', 151, 385, 234)
('Arsenal', 150, 416, 266)
('Manchester United', 145, 364, 219)
('Lazio', 143, 421, 278)
('FC Krasnodar', 139, 296, 157)
('Monaco', 129, 382, 253)
('Marseille', 109, 362, 253)
('RasenBallsport Leipzig', 109, 267, 158)
('Atalanta', 95, 373, 278)
('Bayer Leverkusen', 87, 359, 272)
('Lokomotiv Moscow', 77, 240, 163)
('AC Milan', 74, 336, 262)
('Borussia M.Gladbach', 74, 333, 259)
('Sevilla', 73, 356, 283)
('Valenci

#### Consulta 3

In [25]:
connection = sqlite3.connect('futbol.db')
cursor = connection.cursor()


cursor.execute("""SELECT Ligas.nombre, Max(Points_league.diff) 
                    FROM Ligas,
                    
                        (SELECT Ligas.nombre as nombre, Temporadas.ano, Max(Puntos.points)-Min(Puntos.points) as diff
                        FROM Ligas, Partidos, Temporadas, 

                            (SELECT Partidos.temporada as temporada, Partidos.liga as liga, Partidos.equipo as equipo, Sum(Partidos.puntos) as points
                            FROM Partidos
                            GROUP BY Partidos.temporada, Partidos.liga, Partidos.equipo) as Puntos
                                            
                        WHERE Ligas.lid=Partidos.liga AND Temporadas.tid=Partidos.temporada
                        AND Puntos.liga=Partidos.liga AND Partidos.temporada=Puntos.temporada
                        AND Puntos.equipo= Partidos.equipo
                        GROUP BY Partidos.temporada, Partidos.liga) as Points_league
                    WHERE Ligas.nombre= Points_league.nombre
                    GROUP BY Points_league.nombre
                    ORDER BY Points_league.diff DESC
                    """)
print("Tabla:\n Liga | Numero de equipos distintos")
for row in cursor.fetchall():
    print(row)
connection.close()

Tabla:
 Liga | Numero de equipos distintos
('EPL', 82)
('Ligue_1', 78)
('Serie_A', 76)
('La_liga', 74)
('Bundesliga', 63)
('RFPL', 55)
